# Create an InterActive Graph of NTRL Lab Analysis Results

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

In [5]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

deznrxn = pd.read_excel(r'C:\Users\v.t.flores\Documents\df_zn_drop_2019_cleaned1.xlsx')
deznrxn = deznrxn.dropna()
deznrxn.set_index('DATETIME', inplace=True)
deznrxn_no_neg_inf = deznrxn[deznrxn['Zn_drop']>-10000]

#parameter for Mg and moisture expanded
df_deznNtrl_param  = deznrxn_no_neg_inf[['104_Pb', '104_Zn', '104_Cu', '104_Ni', '104_Co', '104_Mn',
       '104_Cr', '104_Ca', '104_Si', '104_Al', '104_Mg', '104_Fe2','104_pH', '104_ORP']]
#Computing for coeff for Moisture
from sklearn.model_selection import train_test_split
X1 = df_deznNtrl_param.dropna()
y1 = deznrxn_no_neg_inf['Zn_drop'].dropna()
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=0)

from sklearn.linear_model import LinearRegression
lm_Zndrop = LinearRegression()
lm_Zndrop.fit(X1_train,y1_train)

coeff_Zn_drop = pd.DataFrame(zip(lm_Zndrop.coef_, X1.columns), columns=['Zn_drop','Ntrl_param'])

import openpyxl
wb1 = openpyxl.load_workbook(r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\7) July _2020.xlsx', data_only=True)
#####This block of code appends the Analysis Results from 
NTRL_lab = wb1['104PU01']
DeZn_TK05 = wb1['105TK05']
MS_lab = wb1['106TK01']
#update for the targetmonth
currentmonthyr = datetime.date(2020,7,1)
print('Done', datetime.datetime.now())

# generates a dataframe for each column in the Analysis excel file 104PU01
ntrl_Pb_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'E', '104_Pb')
ntrl_Zn_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'F', '104_Zn')
ntrl_Cu_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'G', '104_Cu')
ntrl_Ni_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'H', '104_Ni')
ntrl_Co_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'I', '104_Co')
ntrl_Mn_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'K', '104_Mn')
ntrl_Cr_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'L', '104_Cr')
ntrl_Ca_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'M', '104_Ca')
ntrl_Si_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'N', '104_Si')
ntrl_Al_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'O', '104_Al')
ntrl_Mg_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'P', '104_Mg')
ntrl_Fe2_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'R', '104_Fe2')
ntrl_pH_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'AI', '104_pH')
ntrl_ORP_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'AJ', '104_ORP')

ntrl_PbZn = ntrl_Pb_df.join(ntrl_Zn_df, how='outer')
ntrl_CuNi = ntrl_Cu_df.join(ntrl_Ni_df, how='outer')
ntrl_Co = ntrl_Co_df
ntrl_MnCr = ntrl_Mn_df.join(ntrl_Cr_df, how='outer')
ntrl_CaSi = ntrl_Ca_df.join(ntrl_Si_df, how='outer')
ntrl_AlMg = ntrl_Al_df.join(ntrl_Mg_df, how='outer')
ntrl_pHORP = ntrl_pH_df.join(ntrl_ORP_df, how='outer')
#--------------
ntrl_PbZnCuNi = ntrl_PbZn.join(ntrl_CuNi, how='outer')
ntrl_CoFeMnCr = ntrl_Co.join(ntrl_MnCr, how='outer')
ntrl_CaSiAlMg = ntrl_CaSi.join(ntrl_AlMg, how='outer')
ntrl_Fe2pHORP = ntrl_Fe2_df.join(ntrl_pHORP, how='outer')

ntrl_PbZnCuNiCoFeMnCr = ntrl_PbZnCuNi.join(ntrl_CoFeMnCr, how='outer')
ntrl_CaSiAlMgFe2Fe3pHORP = ntrl_CaSiAlMg.join(ntrl_Fe2pHORP, how='outer')
ntrl_PbZnCuNiCoFeMnCrCaSiAlMgFe2Fe3pHORP = ntrl_PbZnCuNiCoFeMnCr.join(ntrl_CaSiAlMgFe2Fe3pHORP, how='outer')
df_main = ntrl_PbZnCuNiCoFeMnCrCaSiAlMgFe2Fe3pHORP
#--------------
# generates a dataframe for each column in the Analysis excel file 105TK05

DeZn_TK05 = wb1['105TK05']
DeZn_TK05_Pb_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'E', '105TK05_Pb')
DeZn_TK05_Zn_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'F', '105TK05_Zn')
DeZn_TK05_Cu_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'G', '105TK05_Cu')
DeZn_TK05_Ni_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'H', '105TK05_Ni')
DeZn_TK05_Co_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'I', '105TK05_Co')
DeZn_TK05_Fe2_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'J', '105TK05_Fe')
DeZn_TK05_Mn_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'K', '105TK05_Mn')
DeZn_TK05_Cr_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'L', '105TK05_Cr')
DeZn_TK05_Ca_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'M', '105TK05_Ca')
DeZn_TK05_Si_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'N', '105TK05_Si')
DeZn_TK05_Al_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'O', '105TK05_Al')
DeZn_TK05_Mg_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'P', '105TK05_Mg')

DeZn_TK05_pH_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'AF', '105TK05_pH')
DeZn_TK05_ORP_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'AG', '105TK05_ORP')

DeZn_TK05_PS90_df = generate_df_from_analysis(DeZn_TK05, 'B', 'C', 'AP', '105TK05_PS90')
#-----------------------------------------------
#105TK03
DeZn_TK03 = wb1['105TK03']
DeZn_TK03_105TK03_Zn_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'D', '105TK03_Zn')
DeZn_TK03_105TK03_Zn_Change_Ratio_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BB', '105TK03_Zn_Change_Ratio')
DeZn_TK03_105TK05_Zn_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BD', '105TK05_Zn')
DeZn_TK03_106TK01_Zn_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BE', '106TK01_Zn')
DeZn_TK03_106TK01_ZnNi_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BF', '106TK01_Zn/Ni')
DeZn_TK03_106TK01_Zn_DropRate_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BG', '106TK01_Zn_DropRate')
DeZn_TK03_106VE04_Znprcnt_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BH', '106VE04_%Zn')
DeZn_TK03_106TH01_Znprcnt_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BI', '106TH01_%Zn')
DeZn_TK03_104PU01_Ni_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BJ', '104PU01_Ni')
DeZn_TK03_105TK05_Ni_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BK', '105TK05_Ni')
DeZn_TK03_Ni_Loss_TK05_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BL', 'Ni_Loss_TK05')

for i in range(0,len(DeZn_TK03_104PU01_Ni_df)):
    if DeZn_TK03_104PU01_Ni_df['104PU01_Ni'][i] == 0:
        DeZn_TK03_104PU01_Ni_df['104PU01_Ni'][i] = np.nan
        
#105TK05 Ni
for i in range(0,len(DeZn_TK03_105TK05_Ni_df)):
    if DeZn_TK03_105TK05_Ni_df['105TK05_Ni'][i] == 0:
        DeZn_TK03_105TK05_Ni_df['105TK05_Ni'][i] = np.nan

#105TK05 Ni Loss
for i in range(0,len(DeZn_TK03_Ni_Loss_TK05_df)):
    if DeZn_TK03_Ni_Loss_TK05_df['Ni_Loss_TK05'][i] == 0:
        DeZn_TK03_Ni_Loss_TK05_df['Ni_Loss_TK05'][i] = np.nan         
        
#105TK05 Zn
for i in range(0,len(DeZn_TK03_105TK05_Zn_df)):
    if DeZn_TK03_105TK05_Zn_df['105TK05_Zn'][i] == 0:
        DeZn_TK03_105TK05_Zn_df['105TK05_Zn'][i] = np.nan    
        
#106TK01 Zn
for i in range(0,len(DeZn_TK03_106TK01_Zn_df)):
    if DeZn_TK03_106TK01_Zn_df['106TK01_Zn'][i] == 0:
        DeZn_TK03_106TK01_Zn_df['106TK01_Zn'][i] = np.nan      

#-----------------------------------------------

DeZn_FT01 = wb1['105FT01']
FT01A_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'D', 'Zn')
FT01B_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'E', 'Zn')
FT01C_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'F', 'Zn')
FT01D_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'G', 'Zn')
FT01E_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'H', 'Zn')
FT01F_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'I', 'Zn')

FT01A_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'J', 'ntu')
FT01B_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'K', 'ntu')
FT01C_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'L', 'ntu')
FT01D_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'M', 'ntu')
FT01E_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'N', 'ntu')
FT01F_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'O', 'ntu')

FT01A_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'J', 'Zn_dr')
FT01B_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'K', 'Zn_dr')
FT01C_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'L', 'Zn_dr')
FT01D_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'M', 'Zn_dr')
FT01E_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'N', 'Zn_dr')
FT01F_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'O', 'Zn_dr')

A = FT01A_Zn_df.join(FT01A_ntu_df, how='outer').join(FT01A_ZDR_df, how='outer')
A['Filter'] = 'FT01A'
A.reset_index(inplace=True)

B = FT01B_Zn_df.join(FT01B_ntu_df, how='outer').join(FT01B_ZDR_df, how='outer')
B['Filter'] = 'FT01B'
B.reset_index(inplace=True)

C = FT01C_Zn_df.join(FT01C_ntu_df, how='outer').join(FT01C_ZDR_df, how='outer')
C['Filter'] = 'FT01C'
C.reset_index(inplace=True)

D = FT01A_Zn_df.join(FT01D_ntu_df, how='outer').join(FT01D_ZDR_df, how='outer')
D['Filter'] = 'FT01D'
D.reset_index(inplace=True)

E = FT01E_Zn_df.join(FT01E_ntu_df, how='outer').join(FT01E_ZDR_df, how='outer')
E['Filter'] = 'FT01E'
E.reset_index(inplace=True)

F = FT01F_Zn_df.join(FT01F_ntu_df, how='outer').join(FT01F_ZDR_df, how='outer')
F['Filter'] = 'FT01F'
F.reset_index(inplace=True)
AtoF = A.append(B, ignore_index =True).append(C, ignore_index = True).append(D, ignore_index=True).append(E, ignore_index=True).append(F, ignore_index=True)
AtoF = ave_filler(AtoF, 'ntu')
AtoF = ave_filler(AtoF, 'Zn_dr')
AtoF.dropna(inplace=True)

#-----------------------------------------------

#fill the nan values with the average of surrounding values
for i in range(0, len(df_main.columns)):
    #print(i, df_main.columns[i])
    ave_filler(df_main, df_main.columns[i])
df_main = df_main.fillna(method='bfill')

df_main2 = df_main[['104_Pb', '104_Zn', '104_Cu', '104_Ni', '104_Co', '104_Mn',
       '104_Cr', '104_Ca', '104_Si', '104_Al', '104_Mg', '104_Fe2', '104_pH',
       '104_ORP',]]
df_main2 = df_main2.dropna(how='any')

zn_drop_lst = list()
datelist = list()
for j in range(0, len(df_main2)):
    zn_drop = 0
    for i in range(0, len(df_main2.iloc[j])):
        #print(df_main2.iloc[j][i], " * " ,coeff_Zn_drop['Zn_drop'][i])
        temp_product = df_main2.iloc[j][i] * coeff_Zn_drop['Zn_drop'][i]
        zn_drop = zn_drop + temp_product
    zn_drop_new = zn_drop + lm_Zndrop.intercept_
    zn_drop_lst.append(zn_drop_new)
    datelist.append(df_main2.index[j])
zn_drop_and_date_lst = list(zip(datelist, zn_drop_lst))
df_zn_drop_and_date = pd.DataFrame(zn_drop_and_date_lst)
df_zn_drop_and_date = df_zn_drop_and_date.set_index(0)
df_zn_drop_and_date['Zn_drop_Pred'] = df_zn_drop_and_date[1]

# df_zn_drop_and_date is the df that we need here

# generates a dataframe for each column in the excel file
MS_106TK01 = generate_df_from_analysis(MS_lab, 'B', 'C', 'BD', 'Zn_drop_ACT')
pd.to_numeric(MS_106TK01["Zn_drop_ACT"], errors='coerce')
for i in range(1, len(MS_106TK01.Zn_drop_ACT)):
    try:
        if type(MS_106TK01.Zn_drop_ACT[i]) == str:
            print(i, MS_106TK01.Zn_drop_ACT[i])
            MS_106TK01.drop(MS_106TK01.index[i], inplace=True)
            print(i, MS_106TK01.Zn_drop_ACT[i])
    except:
        pass

print('Done ', datetime.datetime.now())
y = df_zn_drop_and_date['Zn_drop_Pred']
MS_y = MS_106TK01["Zn_drop_ACT"]*100

y_Pb_ntrl = df_main['104_Pb']
y_Zn_ntrl = df_main['104_Zn']
y_Cu_ntrl = df_main['104_Cu']
y_Ni_ntrl = df_main['104_Ni']
y_Co_ntrl = df_main['104_Co']
y_Mn_ntrl = df_main['104_Mn']
y_Cr_ntrl = df_main['104_Cr']
y_Ca_ntrl = df_main['104_Ca']
y_Si_ntrl = df_main['104_Si']
y_Al_ntrl = df_main['104_Al']
y_Mg_ntrl = df_main['104_Mg']
y_Fe2_ntrl = df_main['104_Fe2']
y_pH_ntrl = df_main['104_pH']
y_ORP_ntrl = df_main['104_ORP']


#fig1, ax1 = plt.subplots()
#fig1 = plt.gcf()
#plt.xticks(rotation=90, fontsize=8)
#ax2 = ax1.twinx()
#ax1.plot(y.index, y, 'g-', linewidth=1)
#ax2.plot(MS_y.index, MS_y, 'b-', ls='--', linewidth=1)
#plt.xlabel('DATETIME')
#plt.ylabel('Zn Drop Actual')
#plt.legend(['Zn_drop_Act', 'Zn_drop_Pred'])
#plt.title('Zn Drop Trend May 2020')
#fig1.savefig(r'C:\Users\v.t.flores\Documents\Zn_drop_19_May2020_0700h.png', dpi=300, bbox_inches='tight')
#fig2, ax21 = plt.subplots()
#fig2 = plt.gcf()
#xi = list(range(len(y.index[-50:])))
#plt.xticks(xi, y.index[-50:], rotation=90, fontsize= 8)
#ax22 = ax21.twinx()
#ax21.plot(y.index[-50:], y[-50:], 'g-', linewidth=1)
#ax22.plot(MS_y.index[-50:], MS_y[-50:], 'b-', ls='--', linewidth=1)
#plt.ylabel('Zn Drop Actual')
#plt.xlabel('DATETIME')

#plt.legend(['Zn_drop_Act', 'Zn_drop_Pred'])
#plt.title('Zn Drop Trend May 2020 [Closer Look]')
#fig2.savefig(r'C:\Users\v.t.flores\Documents\Zn_drop_19_May2020_zoom_0700h.png', dpi=300, bbox_inches='tight')

#combines the actual and predicted zn drop dataframes
zn_drop_predictions = MS_106TK01.join(df_zn_drop_and_date, how ='outer')



#fcn dfn
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig Add traces to fig
fig.add_trace(go.Scatter(x=y_Zn_ntrl.index, y=y_Zn_ntrl, name="NTRL_Zn"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_Pb_ntrl.index, y=y_Pb_ntrl, name="NTRL_Pb"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_Cu_ntrl.index, y=y_Cu_ntrl, name="NTRL_Cu"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_Ni_ntrl.index, y=y_Ni_ntrl, name="NTRL_Ni"), secondary_y=True,)
fig.add_trace(go.Scatter(x=y_Co_ntrl.index, y=y_Co_ntrl, name="NTRL_Co"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_Mn_ntrl.index, y=y_Mn_ntrl, name="NTRL_Mn"), secondary_y=True,)
fig.add_trace(go.Scatter(x=y_Cr_ntrl.index, y=y_Cr_ntrl, name="NTRL_Cr"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_Ca_ntrl.index, y=y_Ca_ntrl, name="NTRL_Ca"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_Si_ntrl.index, y=y_Si_ntrl, name="NTRL_Si"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_Al_ntrl.index, y=y_Al_ntrl, name="NTRL_Al"), secondary_y=True,)
fig.add_trace(go.Scatter(x=y_Mg_ntrl.index, y=y_Mg_ntrl, name="NTRL_Mg"), secondary_y=True,)
fig.add_trace(go.Scatter(x=y_Fe2_ntrl.index, y=y_Fe2_ntrl, name="NTRL_Fe2"), secondary_y=False,)
fig.add_trace(go.Scatter(x=y_pH_ntrl.index, y=y_pH_ntrl, name="NTRL_pH"), secondary_y=True,)
fig.add_trace(go.Scatter(x=y_ORP_ntrl.index, y=y_ORP_ntrl, name="NTRL_ORP"), secondary_y=True,)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
t = datetime.datetime.now()
title_plot = "<b>NTRL Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title
fig.update_layout(title_text=title_plot)
# Set x-axis title
fig.update_xaxes(title_text="DateTime")
# Set y-axes titles
fig.update_yaxes(title_text="<b>Pb, Zn , Cu, Co, Cr, Ca, Si, Fe2</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Ni, Mn, Al, Mg, pH, ORP</b>", secondary_y=True)
fig.show()

#-------fig2-------
fig2 = make_subplots(specs=[[{"secondary_y": True}]])
# fig2 Add traces
fig2.add_trace(go.Scatter(x=y.index, y=y, name="Zn_drop_Pred"), secondary_y=False,)
fig2.add_trace(go.Scatter(x=MS_y.index, y=MS_y, name="Zn_drop_ACT"),secondary_y=True,)
title_plot2 = "<b>Zn Drop Trend InterActive</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
fig2.update_layout(title_text=title_plot2) # Adds figure title
fig2.update_xaxes(title_text="DateTime") # Set x-axis title
fig2.update_yaxes(title_text="<b>Predicted</b> Zn Drop", secondary_y=False) # Set y-axes titles primary axis
fig2.update_yaxes(title_text="<b>Actual</b> Zn Drop", secondary_y=True)    # Set y-axes titles secondary axis
fig2.show()
#----------------
#------fig3------
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
# fig3 Add traces to fig
#---example of how the add_trace syntax looks like
#fig3.add_trace(go.Scatter(x=DeZn_TK05_Pb_df.index, y=DeZn_TK05_Pb_df['105TK05_Pb'], name="105TK05_Pb"), secondary_y=True,)

generate_add_trace(fig3, DeZn_TK05_Pb_df, True)
generate_add_trace(fig3, DeZn_TK05_Zn_df, True)
generate_add_trace(fig3, DeZn_TK05_Cu_df, True)
generate_add_trace(fig3, DeZn_TK05_Ni_df, False)
generate_add_trace(fig3, DeZn_TK05_Co_df, True)
generate_add_trace(fig3, DeZn_TK05_Fe2_df, False)
generate_add_trace(fig3, DeZn_TK05_Mn_df, False)
generate_add_trace(fig3, DeZn_TK05_Cr_df, True)
generate_add_trace(fig3, DeZn_TK05_Ca_df, True)
generate_add_trace(fig3, DeZn_TK05_Si_df, True)
generate_add_trace(fig3, DeZn_TK05_Al_df, False)
generate_add_trace(fig3, DeZn_TK05_Mg_df, False)
generate_add_trace(fig3, DeZn_TK05_pH_df, False)
generate_add_trace(fig3, DeZn_TK05_ORP_df, False)
generate_add_trace(fig3, DeZn_TK05_PS90_df, False)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plot3 = "<b>105TK05 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
fig3.update_layout(title_text=title_plot3)
fig3.update_xaxes(title_text="DateTime") # Set x-axis title
fig3.update_yaxes(title_text="<b>Pb, Zn , Cu, Co, Cr, Ca, Si, Fe2</b>", secondary_y=False) # Set y-axes titles primary axis
fig3.update_yaxes(title_text="<b>Ni, Mn, Al, Mg, pH, ORP</b>", secondary_y=True) # Set y-axes titles secondary axis
#----------------
#------fig4------
fig4 = make_subplots(specs=[[{"secondary_y": True}]])
# fig4 Add traces to fig
generate_add_trace(fig4, DeZn_TK03_105TK03_Zn_df, True)
generate_add_trace(fig4, DeZn_TK03_105TK03_Zn_Change_Ratio_df, False)
generate_add_trace(fig4, DeZn_TK03_105TK05_Zn_df, True)
generate_add_trace(fig4, DeZn_TK03_106TK01_Zn_df, True)
generate_add_trace(fig4, DeZn_TK03_106TK01_ZnNi_df, True)
generate_add_trace(fig4, DeZn_TK03_106TK01_Zn_DropRate_df, False)
generate_add_trace(fig4, DeZn_TK03_106VE04_Znprcnt_df, True)
generate_add_trace(fig4, DeZn_TK03_106TH01_Znprcnt_df, True)
generate_add_trace(fig4, DeZn_TK03_104PU01_Ni_df, False)
generate_add_trace(fig4, DeZn_TK03_105TK05_Ni_df, False)
generate_add_trace(fig4, DeZn_TK03_Ni_Loss_TK05_df, True)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plot4 = "<b>105TK03 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
fig4.update_layout(title_text=title_plot4)
fig4.update_xaxes(title_text="DateTime") # Set x-axis title
fig4.update_yaxes(title_text="TK03_Zn, TK05_Zn , 106TK01_Zn, 106TK01_Zn/Ni, VE04_Zn%, 106TH01_Zn%, Ni_Loss", secondary_y=False) # Set y-axes titles primary axis
fig4.update_yaxes(title_text="Zn_ChangeRatio, 106TK01_Zn_DropRate, 104PU01_Ni, Mg, 105TK05_Ni, Ni_Loss_TK05", secondary_y=True) # Set y-axes titles secondary axis

#import plotly.express as px
fig5 = px.scatter(AtoF, x="DATETIME", y="Zn_dr", size="Zn", color="Filter", hover_name="ntu", size_max=40)
title_plot5 = "<b>Polishing Filter</b>" + '<br>' + 'Marker Size = Zn conc' + ' ; ' + 'On Hover = NTU'
fig5.update_layout(title_text=title_plot5)


with open(r'\\THGOBNA002\thpal\05 Production\150 DCS\Miscellaneous\InterActive Plots July2020.html', 'w') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig3.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig4.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig5.to_html(full_html=False, include_plotlyjs='cdn'))
    

Done 2020-07-22 15:09:55.558918
Done  2020-07-22 15:10:00.340151
